<a href="https://colab.research.google.com/github/oldliutou/pytorch_study/blob/main/train_gpu_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch


In [ ]:
torch.cuda.is_available()

True

In [ ]:
# dir(torch.FloatTensor)

In [ ]:
help(torch.cuda.is_available)

Help on function is_available in module torch.cuda:

is_available() -> bool
    Returns a bool indicating if CUDA is currently available.



In [ ]:
import time

import  torch
import torchvision
from torch import nn
from torch.utils.tensorboard import SummaryWriter
# from model import  *
# 定义训练的设备
device = torch.device("cuda")

# 准备数据集
from torch.utils.data import DataLoader

train_data = torchvision.datasets.CIFAR10("../dataset",train=True,transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10("../dataset",train=False,transform=torchvision.transforms.ToTensor(),
                                          download=True)

train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"训练数据集的长度为：{train_data_size}")
print(f"测试数据集的长度为：{test_data_size}")

# 利用dataloader 来加载数据集
train_dataloader = DataLoader(train_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

# 创建引入的神经网络

class Tudui(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv2d1 = nn.Conv2d(3,32,5,stride=1,padding=2)
        self.maxpool2d1 = nn.MaxPool2d(kernel_size=2)
        self.conv2d2 = nn.Conv2d(32,32,kernel_size=5,padding=2)
        self.maxpool2d2 = nn.MaxPool2d(kernel_size=2)
        self.conv2d3 = nn.Conv2d(32,64,kernel_size=5,padding=2)
        self.maxpool2d3 = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.liear1 = nn.Linear(1024,64)
        self.liear2 = nn.Linear(64,10)

    def forward(self,x):
        x = self.conv2d1(x)
        x = self.maxpool2d1(x)
        x = self.conv2d2(x)
        x = self.maxpool2d2(x)
        x = self.conv2d3(x)
        x = self.maxpool2d3(x)
        x = self.flatten(x)
        x = self.liear1(x)
        x = self.liear2(x)
        return x

tudui = Tudui()
tudui = tudui.to(device)

# 损失函数

loss_fn = nn.CrossEntropyLoss()

loss_fn = loss_fn.to(device)
# 优化器
learning_rate = 0.01
optimizer = torch.optim.SGD(tudui.parameters(),lr=learning_rate)

# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 0
# 添加tensorboard
writer = SummaryWriter("../logs_train")
start_time = time.time()
for i in range(25):
    print(f"-------------第{(i+1)}轮训练开始-------------")
#     训练步骤开始
    tudui.train()
    for data in train_dataloader:
        imgs,targets = data

        imgs = imgs.to(device)
        targets = targets.to(device)

        output = tudui(imgs)
        loss = loss_fn(output,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step = total_train_step + 1
        if(total_train_step % 100 == 0):
            end_time = time.time()
            print(end_time-start_time)
            print(f"训练次数： {total_train_step},loss {loss.item()}")
            writer.add_scalar("train_loss",loss.item(),total_train_step)
    # 测试步骤开始
    tudui.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            output = tudui(imgs)
            loss = loss_fn(output, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (output.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy
            accuracy_result = torch.true_divide(total_accuracy, test_data_size)
    print(f"整体测试集上的Loss : {total_test_loss}")
    print(f"整体测试集上的正确率 : {accuracy_result}")

    writer.add_scalar("test_loss",total_test_loss,total_test_step)
    writer.add_scalar("test_accuracy",accuracy_result,total_test_step)
    total_test_step = total_test_step + 1

#     保存每一个epoch的模型
    torch.save(tudui,f"tudui_{(i+1)}_gpu.pth")
    print(f"第{(i+1)}轮模型已经保存")
writer.close()

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
-------------第1轮训练开始-------------
1.6732761859893799
训练次数： 100,loss 2.294492483139038
3.142167329788208
训练次数： 200,loss 2.282834053039551
4.684251308441162
训练次数： 300,loss 2.2679522037506104
6.192445516586304
训练次数： 400,loss 2.1869750022888184
7.67684268951416
训练次数： 500,loss 2.044241428375244
9.15436339378357
训练次数： 600,loss 2.0194151401519775
10.671521663665771
训练次数： 700,loss 2.032662868499756
整体测试集上的Loss : 315.7196797132492
整体测试集上的正确率 : 0.2766000032424927
第0轮模型已经保存
-------------第2轮训练开始-------------
14.349559783935547
训练次数： 800,loss 1.9151002168655396
15.856079339981079
训练次数： 900,loss 1.8827039003372192
17.35385012626648
训练次数： 1000,loss 1.9736554622650146
18.83605432510376
训练次数： 1100,loss 1.9867472648620605
20.324360847473145
训练次数： 1200,loss 1.7324024438858032
21.81330156326294
训练次数： 1300,loss 1.714358925819397
23.305447101593018
训练次数： 1400,loss 1.777355670928955
24.814934253692627
